In [1]:
import re
import numpy as np
import pandas as pd
from pprint import pprint


In [2]:
#Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

#Spcay for lemmatization
import spacy


In [4]:
#plotting tools
import pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plt
%matplotlib inline

In [5]:
#Enabling logging for gensim 
import logging
logging.basicConfig(format='%(asctime)s:%(levelname)s:%(message)s',level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [6]:
#stopwords
from nltk.corpus import stopwords
stop_words=stopwords.words('english')
stop_words.extend(['from','subject','re','edu','use'])

In [7]:
#Import dataset
df = pd.read_json('https://raw.githubusercontent.com/selva86/datasets/master/newsgroups.json')
df.head()

,content,target,target_names
0,From: lerxst@wam.umd.edu (where's my thing)\nS...,7,rec.autos
1,From: guykuo@carson.u.washington.edu (Guy Kuo)...,4,comp.sys.mac.hardware
10,From: irwin@cmptrc.lonestar.org (Irwin Arnstei...,8,rec.motorcycles
100,From: tchen@magnus.acs.ohio-state.edu (Tsung-K...,6,misc.forsale
1000,From: dabl2@nlm.nih.gov (Don A.B. Lindbergh)\n...,2,comp.os.ms-windows.misc


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11314 entries, 0 to 9999
Data columns (total 3 columns):
content         11314 non-null object
target          11314 non-null int64
target_names    11314 non-null object
dtypes: int64(1), object(2)
memory usage: 353.6+ KB


In [9]:
df.head(10)

,content,target,target_names
0,From: lerxst@wam.umd.edu (where's my thing)\nS...,7,rec.autos
1,From: guykuo@carson.u.washington.edu (Guy Kuo)...,4,comp.sys.mac.hardware
10,From: irwin@cmptrc.lonestar.org (Irwin Arnstei...,8,rec.motorcycles
100,From: tchen@magnus.acs.ohio-state.edu (Tsung-K...,6,misc.forsale
1000,From: dabl2@nlm.nih.gov (Don A.B. Lindbergh)\n...,2,comp.os.ms-windows.misc
10000,From: a207706@moe.dseg.ti.com (Robert Loper)\n...,7,rec.autos
10001,From: kimman@magnus.acs.ohio-state.edu (Kim Ri...,6,misc.forsale
10002,From: kwilson@casbah.acns.nwu.edu (Kirtley Wil...,2,comp.os.ms-windows.misc
10003,Subject: Re: Don't more innocents die without ...,0,alt.atheism
10004,From: livesey@solntze.wpd.sgi.com (Jon Livesey...,0,alt.atheism


In [10]:
df.target_names.unique()

array(['rec.autos', 'comp.sys.mac.hardware', 'rec.motorcycles',
       'misc.forsale', 'comp.os.ms-windows.misc', 'alt.atheism',
       'comp.graphics', 'rec.sport.baseball', 'rec.sport.hockey',
       'sci.electronics', 'sci.space', 'talk.politics.misc', 'sci.med',
       'talk.politics.mideast', 'soc.religion.christian',
       'comp.windows.x', 'comp.sys.ibm.pc.hardware', 'talk.politics.guns',
       'talk.religion.misc', 'sci.crypt'], dtype=object)

In [11]:
#remove emails and newline characters

#convert to list
data=df.content.values.tolist()

#remove emails
data=[re.sub('\S*@\S*\s?', '', sent) for sent in data]

# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

pprint(data[:1])

['From: (wheres my thing) Subject: WHAT car is this!? Nntp-Posting-Host: '
 'rac3.wam.umd.edu Organization: University of Maryland, College Park Lines: '
 '15 I was wondering if anyone out there could enlighten me on this car I saw '
 'the other day. It was a 2-door sports car, looked to be from the late 60s/ '
 'early 70s. It was called a Bricklin. The doors were really small. In '
 'addition, the front bumper was separate from the rest of the body. This is '
 'all I know. If anyone can tellme a model name, engine specs, years of '
 'production, where this car is made, history, or whatever info you have on '
 'this funky looking car, please e-mail. Thanks, - IL ---- brought to you by '
 'your neighborhood Lerxst ---- ']


In [12]:
#Tokenize words and Clean-up text

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

[['from', 'wheres', 'my', 'thing', 'subject', 'what', 'car', 'is', 'this', 'nntp', 'posting', 'host', 'rac', 'wam', 'umd', 'edu', 'organization', 'university', 'of', 'maryland', 'college', 'park', 'lines', 'was', 'wondering', 'if', 'anyone', 'out', 'there', 'could', 'enlighten', 'me', 'on', 'this', 'car', 'saw', 'the', 'other', 'day', 'it', 'was', 'door', 'sports', 'car', 'looked', 'to', 'be', 'from', 'the', 'late', 'early', 'it', 'was', 'called', 'bricklin', 'the', 'doors', 'were', 'really', 'small', 'in', 'addition', 'the', 'front', 'bumper', 'was', 'separate', 'from', 'the', 'rest', 'of', 'the', 'body', 'this', 'is', 'all', 'know', 'if', 'anyone', 'can', 'tellme', 'model', 'name', 'engine', 'specs', 'years', 'of', 'production', 'where', 'this', 'car', 'is', 'made', 'history', 'or', 'whatever', 'info', 'you', 'have', 'on', 'this', 'funky', 'looking', 'car', 'please', 'mail', 'thanks', 'il', 'brought', 'to', 'you', 'by', 'your', 'neighborhood', 'lerxst']]


In [13]:
#Creating Bigram and Trigram Models
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['from', 'wheres', 'my', 'thing', 'subject', 'what', 'car', 'is', 'this', 'nntp_posting_host', 'rac_wam_umd_edu', 'organization', 'university', 'of', 'maryland_college_park', 'lines', 'was', 'wondering', 'if', 'anyone', 'out', 'there', 'could', 'enlighten', 'me', 'on', 'this', 'car', 'saw', 'the', 'other', 'day', 'it', 'was', 'door', 'sports', 'car', 'looked', 'to', 'be', 'from', 'the', 'late', 'early', 'it', 'was', 'called', 'bricklin', 'the', 'doors', 'were', 'really', 'small', 'in', 'addition', 'the', 'front_bumper', 'was', 'separate', 'from', 'the', 'rest', 'of', 'the', 'body', 'this', 'is', 'all', 'know', 'if', 'anyone', 'can', 'tellme', 'model', 'name', 'engine', 'specs', 'years', 'of', 'production', 'where', 'this', 'car', 'is', 'made', 'history', 'or', 'whatever', 'info', 'you', 'have', 'on', 'this', 'funky', 'looking', 'car', 'please', 'mail', 'thanks', 'il', 'brought', 'to', 'you', 'by', 'your', 'neighborhood', 'lerxst']


In [14]:
#Remove Stopwords, Make Bigrams and Lemmatize
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [15]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)



OSError: [E050] Can't find model 'en'. It doesn't seem to be a shortcut link, a Python package or a valid path to a data directory.

In [17]:
# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])



In [18]:
# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['where', 's', 'thing', 'car', 'nntp_poste', 'host', 'umd', 'organization', 'university', 'maryland_college', 'park', 'line', 'wonder', 'anyone', 'could', 'enlighten', 'car', 'see', 'day', 'door', 'sport', 'car', 'look', 'late', 'early', 'call', 'bricklin', 'door', 'really', 'small', 'addition', 'front_bumper', 'separate', 'rest', 'body', 'know', 'anyone', 'tellme', 'model', 'name', 'engine', 'specs', 'year', 'production', 'car', 'make', 'history', 'info', 'funky', 'look', 'car', 'mail', 'thank', 'bring', 'neighborhood', 'lerxst']]


In [20]:
#Create the Dictionary and Corpus needed for Topic Modeling
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 2), (2, 1), (3, 1), (4, 1), (5, 1), (6, 5), (7, 1), (8, 1), (9, 2), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 2), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 1)]]


In [21]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('addition', 1),
  ('anyone', 2),
  ('body', 1),
  ('bricklin', 1),
  ('bring', 1),
  ('call', 1),
  ('car', 5),
  ('could', 1),
  ('day', 1),
  ('door', 2),
  ('early', 1),
  ('engine', 1),
  ('enlighten', 1),
  ('front_bumper', 1),
  ('funky', 1),
  ('history', 1),
  ('host', 1),
  ('info', 1),
  ('know', 1),
  ('late', 1),
  ('lerxst', 1),
  ('line', 1),
  ('look', 2),
  ('mail', 1),
  ('make', 1),
  ('maryland_college', 1),
  ('model', 1),
  ('name', 1),
  ('neighborhood', 1),
  ('nntp_poste', 1),
  ('organization', 1),
  ('park', 1),
  ('production', 1),
  ('really', 1),
  ('rest', 1),
  ('s', 1),
  ('see', 1),
  ('separate', 1),
  ('small', 1),
  ('specs', 1),
  ('sport', 1),
  ('tellme', 1),
  ('thank', 1),
  ('thing', 1),
  ('umd', 1),
  ('university', 1),
  ('where', 1),
  ('wonder', 1),
  ('year', 1)]]

In [22]:
# Building the Topic Model
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [23]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.048*"mhz" + 0.043*"output" + 0.043*"input" + 0.026*"microsoft" + '
  '0.026*"menu" + 0.025*"function" + 0.025*"lee" + 0.020*"documentation" + '
  '0.017*"toolkit" + 0.016*"borland"'),
 (1,
  '0.021*"god" + 0.019*"say" + 0.017*"people" + 0.017*"christian" + '
  '0.014*"believe" + 0.013*"life" + 0.013*"man" + 0.011*"claim" + '
  '0.010*"child" + 0.010*"word"'),
 (2,
  '0.062*"illinoi" + 0.041*"urbana" + 0.033*"camera" + 0.024*"distance" + '
  '0.016*"feb" + 0.012*"reed" + 0.007*"mapping" + 0.002*"pluto" + '
  '0.000*"laptop" + 0.000*"fox"'),
 (3,
  '0.087*"israel" + 0.063*"israeli" + 0.043*"arab" + 0.033*"international" + '
  '0.030*"command" + 0.028*"gay" + 0.025*"homosexual" + 0.022*"professional" + '
  '0.020*"count" + 0.018*"center"'),
 (4,
  '0.121*"key" + 0.044*"encryption" + 0.035*"font" + 0.031*"clipper" + '
  '0.031*"clipper_chip" + 0.026*"security" + 0.024*"algorithm" + '
  '0.022*"secure" + 0.021*"bit" + 0.020*"proposal"'),
 (5,
  '0.199*"space" + 0.034*"satellite" +

In [24]:
#Compute Model Perplexity and Coherence Score

In [25]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -14.691597835923057

Coherence Score:  0.5215587403547935


In [26]:
#Visualize the topics-keywords
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

C:\Users\M1046091\AppData\Local\Continuum\anaconda3\envs\condaOpenCV\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
16     40.567066        1       1 -0.375139 -0.110852
14     15.810532        1       2 -0.272315  0.026707
1       9.386783        1       3 -0.299546 -0.147071
6       6.375012        1       4 -0.192522 -0.017068
17      5.600935        1       5 -0.145211  0.063700
9       4.639091        1       6 -0.149198  0.022032
12      4.581126        1       7 -0.091193  0.439656
8       4.197324        1       8 -0.149571 -0.075584
15      3.018627        1       9 -0.066146  0.020719
10      0.921590        1      10  0.118877  0.004581
4       0.912183        1      11  0.092553 -0.012244
18      0.809231        1      12  0.149082 -0.013626
3       0.742864        1      13  0.130690 -0.012788
5       0.578489        1      14  0.134338 -0.010390
19      0.399523        1      15  0.174251 -0.024823
11      0.355901        1      16  0.179857 -0.027147
7       0.345039        1      17  0.186076 -0.030010
13      0.336180        1      18  0.180892 -0.027597
0       0.330119        1      19  0.188988 -0.031452
2       0.092384        1      20  0.205235 -0.036744, topic_info=      Category          Freq                Term         Total  loglift  \
term                                                                      
39596  Default  62183.000000                  ax  62183.000000  30.0000   
70584  Default  19422.000000                line  19422.000000  29.0000   
23920  Default  17361.000000        organization  17361.000000  28.0000   
21116  Default  25067.000000                 not  25067.000000  27.0000   
24280  Default   7837.000000          university   7837.000000  26.0000   
14324  Default  15700.000000                  do  15700.000000  25.0000   
40036  Default   4390.000000                 max   4390.000000  24.0000   
45366  Default   6984.000000                 use   6984.000000  23.0000   
32139  Default   5356.000000                year   5356.000000  22.0000   
2629   Default   8946.000000                  go   8946.000000  21.0000   
15937  Default   5112.000000              system   5112.000000  20.0000   
56611  Default   6379.000000                host   6379.000000  19.0000   
47576  Default   7455.000000              people   7455.000000  18.0000   
49258  Default  12235.000000                  be  12235.000000  17.0000   
45440  Default   3724.000000              window   3724.000000  16.0000   
34249  Default  11476.000000             article  11476.000000  15.0000   
67218  Default  10471.000000                 say  10471.000000  14.0000   
16264  Default   2793.000000                file   2793.000000  13.0000   
25780  Default  14500.000000               write  14500.000000  12.0000   
40481  Default  13988.000000               would  13988.000000  11.0000   
57945  Default   3303.000000                card   3303.000000  10.0000   
41142  Default   1823.000000               space   1823.000000   9.0000   
42600  Default   2870.000000                game   2870.000000   8.0000   
2913   Default   4594.000000             problem   4594.000000   7.0000   
58333  Default   3094.000000         information   3094.000000   6.0000   
29985  Default   2725.000000                team   2725.000000   5.0000   
51045  Default   1744.000000                 key   1744.000000   4.0000   
64664  Default   3560.000000                 day   3560.000000   3.0000   
13001  Default   3909.000000                 run   3909.000000   2.0000   
3910   Default   3246.000000               drive   3246.000000   1.0000   
...        ...           ...                 ...           ...      ...   
40449  Topic20      0.016554              laptop      1.167814   2.7307   
62500  Topic20      0.016550               darin      1.167786   2.7305   
49137  Topic20      0.016550                 fox      1.167789   2.7305   
3533   Topic20      0.016549           cubs_suck      1.167811   2.73

In [ ]:
#Building LDA Mallet Model
